In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [2]:
# 读取数据
from mxnet.gluon import data as gdata

In [3]:
batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [4]:
for X, y in data_iter:
    print(X, y)
    break


[[-0.64965743  1.37336445]
 [ 1.1480552  -0.25492391]
 [-0.5013274  -1.67310596]
 [ 0.18896957 -0.16971196]
 [-0.51129895  0.46543437]
 [-1.12205899  2.88352633]
 [-0.60309178  0.18327411]
 [ 0.52092004  0.18693134]
 [-0.40661776  0.80200833]
 [ 0.27814385  0.09634511]]
<NDArray 10x2 @cpu(0)> 
[-1.76639521  7.37199306  8.87740326  5.15670633  1.61399984 -7.82870817
  2.36237001  4.61766386  0.64714241  4.42859745]
<NDArray 10 @cpu(0)>


In [5]:
# 定义模型
from mxnet.gluon import nn

In [6]:
net = nn.Sequential()
net.add(nn.Dense(1))

In [7]:
# 初始化模型参数
from mxnet import init

In [8]:
net.initialize(init.Normal(sigma=0.01))

In [9]:
# 定义损失函数
from mxnet.gluon import loss as gloss

In [10]:
loss = gloss.L2Loss()

In [11]:
# 定义优化算法
from mxnet import gluon

In [12]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [14]:
# 训练模型
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.040605
epoch 2, loss: 0.000159
epoch 3, loss: 0.000050


In [15]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.9998821  -3.39976406]]
 <NDArray 1x2 @cpu(0)>)

In [16]:
true_b, dense.bias.data()

(4.2, 
 [ 4.1996932]
 <NDArray 1 @cpu(0)>)